# 이지어드민

- 수집을 최근 30일로
- 저장은 나중에

In [14]:
import requests
import psycopg2
from datetime import datetime, timedelta
import time
import pandas as pd

In [6]:
def get_order_info(start_date, end_date):


    partner_key = '73c6ac2e3366905793ab35d5a222d572'
    domain_key = '73c6ac2e33669057718abd505d76aa87366d3eed6643997457a0f9a03797aa9f'
   
    limit = '100'
    

    url = f'http://api2.cloud.ezadmin.co.kr/ezadmin/function.php?partner_key={partner_key}&domain_key={domain_key}&action=get_order_info&date_type=collect_date&start_date={start_date}&end_date={end_date}&limit=1000'
    res = requests.get(url)

    if res.status_code == 200:
        return res.json()['data']
    else:
        return ValueError

In [ ]:
def save_order_info(data_list):
    pass

In [7]:
test = get_order_info('2023-06-01', '2023-06-01')

In [8]:
len(test)

380

In [12]:
data_list = [list(d.values()) for d in test]

In [15]:
df = pd.DataFrame(data_list)

In [17]:
df.to_clipboard()

In [11]:
test[1].values()

dict_values(['0', '503100', '10053', '쿠팡', '11000181774027', '4935975581', '', '7246571242', '써큐시안 블러드케어 60캡슐', '써큐시안 블러드케어 - 초임계 식물성 알티지 오메가3 콜레스테롤 혈행 홍국 코큐텐 6종 주원료 13가지 복합 기능성, 2+1개', '1', '서정임', '0502-1767-3028', '0502-1767-3028', '서정임', '0502-1767-3028', '0502-1767-3028', '경기도 고양시 일산서구 탄현동 1480 탄현마을7단지아파트 704동1510호', '10249', '문 앞', '8', '0', '2023-06-01 09:04:10', '2023-05-31 14:44:14', '2023-06-01 12:16:04', '2023-06-01 12:20:49', '2023-06-01 09:31:17', '134268', '151990', '0', '30003', '575524352676', '선불', '0', '', '0', '0', '7246571242', 'ju*****@ha', [{'prd_seq': '543106', 'order_cs': '0', 'product_id': '00051', 'link_id': '', 'new_link_id': '', 'name': '써큐시안 블러드케어, 1개', 'options': '', 'qty': '3', 'enable_sale': '1', 'supply_code': '20001', 'supply_name': '자사', 'brand': '', 'supply_options': '', 'barcode': '00051', 'shop_price': '0', 'prd_supply_price': '134268', 'prd_amount': '151990', 'extra_money': '0', 'is_gift': '0', 'cancel_date': '', 'change_date': ''}, {'prd_seq': '54

# 네이버

- 테스트로 테이블을 하나 만들어서 데이터 검증 필요
    - 2023 한달 단위로 비교
- 최근 30일로 만들기

In [18]:
import requests
import bcrypt
import pybase64
import time
import urllib.request
import urllib.parse


def get_token(client_id, client_secret, type_="SELF") -> str:
    timestamp = str(int((time.time()-3) * 1000))
    pwd = f'{client_id}_{timestamp}'
    hashed = bcrypt.hashpw(pwd.encode('utf-8'), client_secret.encode('utf-8'))
    client_secret_sign = pybase64.standard_b64encode(hashed).decode('utf-8')

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data_ = {
        "client_id": client_id,
        "timestamp": timestamp,
        "client_secret_sign": client_secret_sign,
        "grant_type": "client_credentials",
        "type": type_
    }

    query = urllib.parse.urlencode(data_)
    url = 'https://api.commerce.naver.com/external/v1/oauth2/token?' + query
    res = requests.post(url=url, headers=headers)
    res_data = res.json()

    while True:
        if 'access_token' in res_data:
            token = res_data['access_token']
            return token
        else:
            print(f'[{res_data}] 토큰 요청 실패')
            time.sleep(1)

token = get_token(client_id='6rmQh0T5jWd9KuJeHIlzU0', client_secret='$2a$04$KJqyFL0J3K5XJdxuTkKZFu')
print(f'발급된 토큰 : ', token)

발급된 토큰 :  rErW5IXzlaYH7zXVALJeU


In [25]:
def get_new_order_list():
    from datetime import datetime, timedelta

    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/last-changed-statuses'
    
    now = datetime.now()
    before_date = now - timedelta(days=2) #이틀전
    iosFormat = before_date.astimezone().isoformat()

    params = {
            'lastChangedFrom' : iosFormat, #조회시작일시
        }

    res = requests.get(url=url, headers=headers, params=params)
    res_data = res.json()

    if 'data' not in res_data: #조회된 정보가 없을 경우 data키 없음
        print('주문 내역 없음')
        return False

    data_list = res_data['data']['lastChangeStatuses']

    for data in data_list:
        print(data) #주문 정보

    return data_list

data = get_new_order_list()

{'lastChangedDate': '2023-06-10T14:40:31.0+09:00', 'orderId': '2023060578638651', 'receiverAddressChanged': False, 'productOrderId': '2023060557429341', 'productOrderStatus': 'PURCHASE_DECIDED', 'paymentDate': '2023-06-05T22:07:05.0+09:00', 'lastChangedType': 'PURCHASE_DECIDED'}
{'lastChangedDate': '2023-06-10T14:40:32.0+09:00', 'orderId': '2023060569736721', 'receiverAddressChanged': False, 'productOrderId': '2023060543462391', 'productOrderStatus': 'PURCHASE_DECIDED', 'paymentDate': '2023-06-05T14:30:17.0+09:00', 'lastChangedType': 'PURCHASE_DECIDED'}
{'lastChangedDate': '2023-06-10T14:43:58.0+09:00', 'orderId': '2023060569465901', 'receiverAddressChanged': False, 'productOrderId': '2023060543027421', 'productOrderStatus': 'PURCHASE_DECIDED', 'paymentDate': '2023-06-05T14:19:12.0+09:00', 'lastChangedType': 'PURCHASE_DECIDED'}
{'lastChangedDate': '2023-06-10T14:59:43.0+09:00', 'orderId': '2023060575282151', 'receiverAddressChanged': False, 'productOrderId': '2023060552225791', 'produc

In [27]:
df = pd.DataFrame(data)

In [37]:
df['orderId'].duplicated(keep=False).unique()

array([False,  True])

In [36]:
df['productOrderId'].duplicated(keep=False).unique()

array([False])

In [42]:
df['productOrderId'].to_list()

['2023060557429341',
 '2023060543462391',
 '2023060543027421',
 '2023060552225791',
 '2023060546398881',
 '2023061039533531',
 '2023060539740591',
 '2023060539879251',
 '2023060552990721',
 '2023060910313211',
 '2023060528214081',
 '2023060919837491',
 '2023060925434621',
 '2023060666508641',
 '2023060557530431',
 '2023060920106001',
 '2023060860679431',
 '2023060534456581',
 '2023060540711821',
 '2023061042816031',
 '2023060853347711',
 '2023061043127981',
 '2023060916471131',
 '2023060726695611',
 '2023061032685611',
 '2023061035682661',
 '2023061037038151',
 '2023061027584591',
 '2023061028193371',
 '2023061029896541',
 '2023061030998561',
 '2023061032911811',
 '2023060989330781',
 '2023061027145171',
 '2023061031794081',
 '2023060921469971',
 '2023060525751901',
 '2023061034958961',
 '2023061035791031',
 '2023061041368541',
 '2023060921628911',
 '2023060675762671',
 '2023060712435521',
 '2023061032308051',
 '2023061040070891',
 '2023060917975521',
 '2023061042564721',
 '20230604223

In [44]:
def get_order_detail():
    import datetime

    headers = {'Authorization': token}
    url = 'https://api.commerce.naver.com/external/v1/pay-order/seller/product-orders/query'
    
    params = {
        'productOrderIds' : df['productOrderId'].to_list()
    }

    res = requests.post(url=url, headers=headers, json=params)
    res_data = res.json()
    
    # print(res_data)
    if 'data' not in res_data:
        return False
    
    for data in res_data['data']:
        for d in data.keys():
            for d2 in data[d].keys():
                print(f'{d2} : {data[d][d2]}')


get_order_detail()

expectedDeliveryMethod : DELIVERY
takingAddress : {'baseAddress': '경기도 김포시 통진읍', 'zipCode': '10016', 'addressType': 'DOMESTIC', 'detailedAddress': '검암2로 68 코린트', 'tel1': '02-3431-9617', 'tel2': '010-2103-1181', 'isRoadNameAddress': True, 'name': '코린트 반품'}
commissionRatingType : 결제수수료
commissionPrePayStatus : GENERAL_PRD
paymentCommission : 5768
expectedSettlementAmount : 149954
inflowPath : 검색>쇼핑검색(네이버쇼핑)
channelCommission : 0
productImediateDiscountAmount : 197100
productProductDiscountAmount : 1000
sellerBurdenImediateDiscountAmount : 197100
sellerBurdenProductDiscountAmount : 1000
knowledgeShoppingSellingInterlockCommission : 3178
quantity : 1
mallId : ncp_1nrk6s_01
productOrderId : 2023053159736641
productClass : 단일상품
productOrderStatus : PURCHASE_DECIDED
productName : [2+1] 써큐시안 블러드케어 홍국 콜레스테롤 혈행개선에 도움 혈압 복합기능성 3구 선물세트
productId : 7362546798
itemNo : 7362546798
placeOrderStatus : OK
decisionDate : 2023-06-11T01:38:35.0+09:00
unitPrice : 357000
optionPrice : 0
productDiscountAmount

# 쿠팡

In [ ]:
import os
import time
import hmac, hashlib
import urllib.parse
import urllib.request
import ssl
import json
import pandas as pd
import datetime

def 쿠팡_주문수집(createdAtFrom, createdAtTo):
    

    os.environ['TZ'] = 'GMT+0'
    date=(datetime.datetime.now() - datetime.timedelta(hours = 9)).strftime('%y%m%dT%H%M%SZ')
    method = "GET"
    print(date)
    path = f"/v2/providers/openapi/apis/api/v4/vendors/A00197911/ordersheets"
    query = urllib.parse.urlencode({"createdAtFrom": createdAtFrom, "createdAtTo": createdAtTo, "status": "ACCEPT"})
    message = date+method+path+query


    accesskey = "73814036-d7f4-4d56-a254-35908249a0c6"
    secretkey = "86ecbb8dcd4f4d9c4bd83f724ed4bfb4919ba0cc"

    #********************************************************#
    #authorize, demonstrate how to generate hmac signature here
    signature=hmac.new(secretkey.encode('utf-8'),message.encode('utf-8'),hashlib.sha256).hexdigest()
    authorization  = "CEA algorithm=HmacSHA256, access-key="+accesskey+", signed-date="+date+", signature="+signature
    #print out the hmac key
    #print(authorization)
    #********************************************************#

    # ************* SEND THE REQUEST *************
    url = "https://api-gateway.coupang.com"+path+"?%s" % query

    req = urllib.request.Request(url)

    req.add_header("Content-type","application/json;charset=UTF-8")
    req.add_header("Authorization",authorization)

    req.get_method = lambda: method

    ctx = ssl.create_default_context()
    ctx.check_hostname = False
    ctx.verify_mode = ssl.CERT_NONE

    try:
        resp = urllib.request.urlopen(req,context=ctx)
    except urllib.request.HTTPError as e:
        print(e.code)
        print(e.reason)
        print(e.fp.read())
    except urllib.request.URLError as e:
        print(e.errno)
        print(e.reason)
        print(e.fp.read())
    else:
        # 200
        body = resp.read().decode(resp.headers.get_content_charset())
        return body
   

In [ ]:
시작일 = datetime.datetime.today().strftime("%Y-%m-%d") # 오늘날짜
종료일 = datetime.datetime.today().strftime("%Y-%m-%d") # 오늘날짜


test = 쿠팡_주문수집(시작일, 종료일)
tt = json.loads(test)